In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

Let’s read the data with the required encoding

In [2]:
path_to_file = '5_data.csv.zip'

In [3]:
retail = pd.read_csv(path_to_file, encoding ='ISO-8859-1', compression='zip')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
retail_columns = retail.columns
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Check if there are duplicate observations in the data

In [5]:
len(retail) - len(retail.drop_duplicates())

5268

In [6]:
retail = retail.drop_duplicates()

The data contains records of both successful and canceled transactions. If a user canceled an order, the transaction number (*InvoiceNo*) starts with a *C* (canceled).

How many transactions were canceled by users?

In [7]:
retail.InvoiceNo.str[0]

0         5
1         5
2         5
3         5
4         5
         ..
541904    5
541905    5
541906    5
541907    5
541908    5
Name: InvoiceNo, Length: 536641, dtype: object

In [8]:
retail.InvoiceNo.str[0].unique()

array(['5', 'C', 'A'], dtype=object)

In [9]:
retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

(9251, 8)

Now filter the data and keep only those orders in the *retail* dataset where *Quantity > 0*.

In [10]:
retail = retail.loc[retail.Quantity > 0]

In [11]:
retail.shape

(526054, 8)

Count the number of orders for each customer *(CustomerID)* from Germany. Keep only those who have made more than N transactions *(InvoiceNo)*, where N is the 80th percentile

In [12]:
invoice_dist = (retail
                .loc[retail.Country == 'Germany']
                .groupby('CustomerID', as_index=False)
                .agg(Total_Invoices=('InvoiceNo', 'nunique')))

In [13]:
invoice_dist.head()

,CustomerID,Total_Invoices
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7


In [14]:
N = invoice_dist.Total_Invoices.quantile(q=0.8)
N

7.0

In [15]:
germany_top = invoice_dist.loc[invoice_dist.Total_Invoices > N].CustomerID

In [16]:
germany_top 

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

Filter the observations and keep only the records for the users we are interested in

In [17]:
top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

In [18]:
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


Which product was added to the cart most frequently, excluding *POST*?

In [19]:
(top_retail_germany
 .groupby('StockCode')
 .size()
 .sort_values(ascending = False))

StockCode
POST     213
22326     52
22328     38
22423     34
20719     30
        ... 
22563      1
22569      1
22574      1
22576      1
22662      1
Length: 1158, dtype: int64

Create a column *Revenue* with the purchase amount, using the columns *Quantity* and *UnitPrice*.

In [20]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [21]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


For each transaction *(InvoiceNo)*, calculate the final order amount

In [22]:
top_revenue = (retail
               .groupby('InvoiceNo', as_index = False)
               .agg(TotalRevenue = ('Revenue', 'sum'))
               .sort_values('TotalRevenue', ascending=False).head())
top_revenue

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00


In [23]:
top_revenue.index

Index([20689, 2202, 17582, 18251, 9034], dtype='int64')